In [1]:
from ultralytics import YOLO
import numpy as np
import os
import torch
from sklearn.model_selection import train_test_split
import shutil

Для распознования и классификации объектов буду использовать модель YOLO версии 8n(nano). Она хорошо подходит под задачу и нано вариант позволит быстро обучить модель, особенно с учетом того, что данных не такое большое количество.

Для разметки данных я буду пользоваться инструментом YAT. В нём можно произвести разметку для YOLO, которая мне и нужна.

In [61]:
images_path = r'C:\Users\Илья\Documents\nanonano\module_d\data\pictures'
lables_path = r'C:\Users\Илья\Documents\nanonano\module_d\data\lables'
print('Размер датасета с картинками:', len(os.listdir(images_path)))
print('Размер датасета с лейблами:', len(os.listdir(images_path)))

Размер датасета с картинками: 161
Размер датасета с лейблами: 161


In [62]:
out_dir = r'C:\Users\Илья\Documents\nanonano\module_d\data'

# для тренировочного набора
os.makedirs(os.path.join(out_dir, 'train/images'), exist_ok=True) 
os.makedirs(os.path.join(out_dir, 'train/labels'), exist_ok=True) 
# для тестового набора
os.makedirs(os.path.join(out_dir, 'test/images'), exist_ok=True) 
os.makedirs(os.path.join(out_dir, 'test/labels'), exist_ok=True) 
# для валидационного набора
os.makedirs(os.path.join(out_dir, 'valid/images'), exist_ok=True) 
os.makedirs(os.path.join(out_dir, 'valid/labels'), exist_ok=True)

In [63]:
images = os.listdir(images_path)

#  3 выборки: 70% train, 15% valid и 15% test
train_img, test_img = train_test_split(images, test_size=0.3, random_state=42)
val_img, test_img = train_test_split(test_img, test_size=0.5, random_state=42)

In [64]:
def files_copy(files, img, labels, out_img, out_labels):
    for file in files:
        shutil.copy(os.path.join(img, file), os.path.join(out_img, file))
        label_file = os.path.splitext(file)[0] + '.txt'
        shutil.copy(os.path.join(labels, label_file), os.path.join(out_labels, label_file))

    print(len(os.listdir(out_img)))



In [65]:
files_copy(train_img, images_path, lables_path, os.path.join(out_dir, 'train/images'), os.path.join(out_dir, 'train/labels'))
files_copy(test_img, images_path, lables_path, os.path.join(out_dir, 'test/images'), os.path.join(out_dir, 'test/labels'))
files_copy(val_img, images_path, lables_path, os.path.join(out_dir, 'valid/images'), os.path.join(out_dir, 'valid/labels'))

153
45
45


In [66]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [67]:
model = YOLO('yolov8s.pt')

In [68]:
import os
os.getcwd()

'c:\\Users\\Илья\\Documents\\nanonano\\module_d'

In [70]:
results = model.train(
   data='data.yaml',
   imgsz=640,
   epochs=4,
   batch=-1,
   device = device,
   cos_lr = True,
   lr0 = 0.01,
   name='yolov8s_4epo'
)

engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=4, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_4epo, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, resume=False, retina_masks=False, save=True, save_

train: Scanning C:\Users\Илья\Documents\nanonano\module_d\data\train\labels.cache... 152 images, 1 backgrounds, 0 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
WARNING AutoBatch: intended for CUDA devices, using default batch-size 16
train: Fast image access  (ping: 0.10.0 ms, read: 1842.3504.5 MB/s, size: 577.1 KB)



train: Scanning C:\Users\Илья\Documents\nanonano\module_d\data\train\labels.cache... 152 images, 1 backgrounds, 0 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.10.0 ms, read: 1966.7454.1 MB/s, size: 579.2 KB)


...

        4/4         0G     0.6143      1.306     0.9852         14        640: 100%|██████████| 10/10 [02:35<00:00, 15.58s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:15<00:00,  7.84s/it]

                   all         45         45      0.945      0.953      0.995      0.912



4 epochs completed in 0.197 hours.
Optimizer stripped from c:\Users\\Documents\nanonano\runs\detect\yolov8s_4epo\weights\last.pt, 22.5MB
Optimizer stripped from c:\Users\\Documents\nanonano\runs\detect\yolov8s_4epo\weights\best.pt, 22.5MB

Validating c:\Users\\Documents\nanonano\runs\detect\yolov8s_4epo\weights\best.pt...
Ultralytics 8.3.127  Python-3.12.1 torch-2.7.0+cpu CPU (AMD Ryzen 5 7520U with Radeon Graphics)
Model summary (fused): 72 layers, 11,128,293 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:15<00:00,  7.95s/it]


                   all         45         45      0.946      0.952      0.995      0.912
                banana          7          7      0.925          1      0.995      0.923
                carrot          7          7          1       0.68      0.995      0.735
                  cube         13         13      0.959          1      0.995      0.892
              cylinder          9          9      0.959          1      0.995      0.951
                   key          3          3      0.879          1      0.995      0.995
               pyramid          2          2          1      0.984      0.995      0.895
                sphere          4          4      0.899          1      0.995      0.995
Speed: 2.4ms preprocess, 281.8ms inference, 0.0ms loss, 48.0ms postprocess per image
Results saved to c:\Users\\Documents\nanonano\runs\detect\yolov8s_4epo
